## Импортируемые модули

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from statistics import mode
import time
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from copy import copy
from tqdm import tqdm
from sklearn.base import BaseEstimator
import os
from sklearn.model_selection import KFold

## Алгоритм

In [20]:
def write(name_dataset, X, y):
    """
    Запись обучающей выборки в отдельный файл
    """
    with open(name_dataset, 'w') as f:
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                f.write(str(int(X[i][j])))
                f.write(' ')
            f.write(str(y[i]) + ' ')
            f.write('\n')

In [19]:
class A1_Rg(BaseEstimator):
    def __init__(self, eps=0.5, min_samples=5):
        self.eps = eps
        self.min_samples = min_samples
        self.cluster_labels_ = None
        self.cluster_classifiers_ = None
        self.classes = None
        self.X_train = None
        self.y_train = None
        
    def fit(self, X, y):
        # Кластеризация с помощью алгоритма DBSCAN
        self.X_train = X
        self.y_train = y
        clustering = DBSCAN(eps=self.eps, min_samples=self.min_samples).fit(X)
        self.cluster_labels_ = clustering.labels_ + 1
        self.classes = np.unique(self.cluster_labels_)
        
        write('Train', X, self.cluster_labels_) # записываем выборку в файл

        # Классификация по кластерам
        self.cluster_classifiers_ = []
        for K in np.unique(self.cluster_labels_):
            print("       class =", K, "     ") 
            mask = self.cluster_labels_ == K
            cluster_X = X[mask]
            with open('params.hpp', 'w') as f: # записываем параметры для компиляции
                alll = X.shape[0]
                f.write("using namespace std;\n")
                f.write("const int all_gl = " + str(alll) + ';\n')
                f.write("const int n = " + str(X.shape[1]) + ';\n')
                f.write("const string cur_class = " + '"' + str(K) + '"' + ';\n')
                f.write("const int m = " + str(X[self.cluster_labels_ != K].shape[0]) + ';\n')
            !g++ -std=c++17 -O3 CVP.cpp -o CVP # компилируем
            !./CVP Train # запускаем
        os.remove("output.txt") # удаляем ненужные файлы
        os.remove("Train")

    def predict(self, X):
        '''
        y_pred = []
        for x in X:
            # Определение кластера, к которому относится объект
            cluster_idx = self.cluster_labels_[np.argmin(np.sum((X - x) ** 2, axis=1))]
            
            # Голосование по тупиковым представительным элементарным классификаторам
            votes = [cls[1] for cls in self.cluster_classifiers_[cluster_idx]]
            majority_class = mode(votes)
            y_pred.append(majority_class)
        
        return np.array(y_pred)
        '''
        counter = np.zeros((X.shape[0], self.classes.shape[0]))
        for i, K in enumerate(self.classes): 
            with open(str(K) + '_CVP', 'r') as f:
                k = 0
                for line in f:
                    
                    l = line.strip().split(' ')
                    l = np.array(list(map(int, l)))
                    mask = l != -1
                    if np.any(np.all(l[np.newaxis, mask] == self.X_train[:, mask], axis=1)):
                        k += 1
                        # print(counter[:, i].shape)
                        # print(l[np.newaxis, l != -1].shape)
                        # print(l.shape, X.shape)
                        # print(X[:, l != -1].shape)
                        counter[:, i] += np.all(l[np.newaxis, mask] == X[:, mask], axis=1)
                counter[:, i] /= k
        y_pred = np.argmax(counter, axis=1)
        # print(y_pred.shape)
        # print((counter == counter[np.arange(counter.shape[0]), y_pred][:, None]))
        mask = (counter == counter[np.arange(counter.shape[0]), y_pred][:, None])
        #print(mask)
        mask = np.sum(mask, axis=1)
        mask = (mask >= 2)
        y_pred = self.classes[y_pred]
        y_pred[mask] = -100
        # print(y_pred.shape)
        y_predr = np.zeros(y_pred.shape).astype(float)
        for i, obj in enumerate(X):
            K = y_pred[i]
            if K == -100:
                cur_X_train = self.X_train
                cur_y_train = self.y_train
            else:
                cur_X_train = self.X_train[self.cluster_labels_ == K]
                cur_y_train = self.y_train[self.cluster_labels_ == K]
            dist = np.sum(cur_X_train != obj, axis=1)
            dist = 1 / (dist + 1e-5)
            dist /= np.sum(dist)
            y_predr[i] = np.sum(cur_y_train * dist)
        return y_predr

    def predict_proba(self, X):
        y_proba = []
        for x in X:
            # Определение кластера, к которому относится объект
            cluster_idx = self.cluster_labels_[np.argmin(np.sum((X - x) ** 2, axis=1))]

            # Усреднение целевой переменной по всем объектам кластера
            cluster_y = [cls[1] for cls in self.cluster_classifiers_[cluster_idx]]
            mean_class = np.mean(cluster_y)
            y_proba.append(mean_class)

        return np.array(y_proba)

## Эксперименты

Предложенный алгоритм был протестирован на 4 датасетах из ресурса UCI: Automobile, Computer Hardware, Servo, Yacht. Основная цель экспериментов - сравнить полученную регрессионную модель с классическими методами восстановления регрессии, такими как линейная регрессия, градиентный бустинг, kNN-регрессия, случайный лес и регрессия опорных векторов.

## Automobile

Этот датасет состоит из из трех типов объектов: (а) спецификация автомобиля с точки зрения различных характеристик, (б) присвоенный ему рейтинг страхового риска, (в) его нормированные потери при использовании по сравнению с другими автомобилями. 

Описание признака:            Принимаемые значения
---------------             -----------
1. symboling:                -3, -2, -1, 0, 1, 2, 3.
  2. normalized-losses:        целое число в промежутке от 65 до 256.
  3. make:                     alfa-romero, audi, bmw, chevrolet, dodge, honda,
                               isuzu, jaguar, mazda, mercedes-benz, mercury,
                               mitsubishi, nissan, peugot, plymouth, porsche,
                               renault, saab, subaru, toyota, volkswagen, volvo
  4. fuel-type:                diesel, gas.
  5. aspiration:               std, turbo.
  6. num-of-doors:             four, two.
  7. body-style:               hardtop, wagon, sedan, hatchback, convertible.
  8. drive-wheels:             4wd, fwd, rwd.
  9. engine-location:          front, rear.
 10. wheel-base:               вещественное число в промежутке от 86.6 до 120.9.
 11. length:                   вещественное число в промежутке от 141.1 до 208.1.
 12. width:                    вещественное число в промежутке от 60.3 до 72.3.
 13. height:                   вещественное число в промежутке от 47.8 до 59.8.
 14. curb-weight:              целое число в промежутке от 1488 до 4066.
 15. engine-type:              dohc, dohcv, l, ohc, ohcf, ohcv, rotor.
 16. num-of-cylinders:         eight, five, four, six, three, twelve, two.
 17. engine-size:              целое число в промежутке от 61 до 326.
 18. fuel-system:              1bbl, 2bbl, 4bbl, idi, mfi, mpfi, spdi, spfi.
 19. bore:                     вещественное число в промежутке от 2.54 до 3.94.
 20. stroke:                   вещественное число в промежутке от 2.07 до 4.17.
 21. compression-ratio:        целое число в промежутке от 7 до 23.
 22. horsepower:               целое число в промежутке от 48 до 288.
 23. peak-rpm:                 целое число в промежутке от 4150 до 6600.
 24. city-mpg:                 целое число в промежутке от 13 до 49.
 25. highway-mpg:              целое число в промежутке от 16 до 54.
 26. price:                    целое число в промежутке от 5118 до 45400.


In [13]:
data = pd.read_csv('imports-85.data', header=None)
data

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845
201,-1,95,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045
202,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485
203,-1,95,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470


In [14]:
for i, col in enumerate(data.columns[:-1]):
    print(data[col].dtype)

int64
object
object
object
object
object
object
object
object
float64
float64
float64
float64
int64
object
object
int64
object
object
object
float64
object
object
int64
int64


Далее кодируем вещественные признаки в категориальные, приводя их к целочисленным значениям, и удаляем колонки с большим количеством уникальных значений, которые могут быть бесполезными для анализа.

Для кодирования вещественных признаков применяется процедура деления на интервалы. Выбирается 5 порогов для признака col, делящих обучающую выборку по этому признаку на 5 равных частей, после чего вещественные значения заменяются на категориальные интервалы.

In [15]:
for i, col in enumerate(data.columns[:-1]):
    if data[col].dtype == 'float64':
        min_v = min(data[col])
        max_v = max(data[col])
        length = (max_v - min_v) / 5
        data[col] = data[col].apply(lambda x: round((x - min_v) / length))
    elif np.unique(data[col]).shape[0] > 22:
        data.drop(col, axis=1, inplace=True)

Удалим объекты с пропущенными значениями из выборки.

In [16]:
dropped = []
for i in range(data.shape[0]):
    if data.iloc[i, -1] == '?':
        dropped.append(i)
# dropped
data.drop(dropped, axis=0, inplace=True)
data.shape

(201, 17)

In [17]:
data

,0,2,3,4,5,6,7,8,9,10,11,12,14,15,17,20,25
0,3,alfa-romero,gas,std,two,convertible,rwd,front,0,2,2,0,dohc,four,mpfi,1,13495
1,3,alfa-romero,gas,std,two,convertible,rwd,front,0,2,2,0,dohc,four,mpfi,1,16500
2,1,alfa-romero,gas,std,two,hatchback,rwd,front,1,2,2,2,ohcv,six,mpfi,1,16500
3,2,audi,gas,std,four,sedan,fwd,front,2,3,2,3,ohc,four,mpfi,1,13950
4,2,audi,gas,std,four,sedan,4wd,front,2,3,3,3,ohc,five,mpfi,0,17450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,volvo,gas,std,four,sedan,rwd,front,3,4,4,3,ohc,four,mpfi,1,16845
201,-1,volvo,gas,turbo,four,sedan,rwd,front,3,4,4,3,ohc,four,mpfi,1,19045
202,-1,volvo,gas,std,four,sedan,rwd,front,3,4,4,3,ohcv,six,mpfi,1,21485
203,-1,volvo,diesel,turbo,four,sedan,rwd,front,3,4,4,3,ohc,six,idi,5,22470


Закодируем категориальные признаки в числовые значения с помощью LabelEncoder, разделим данные на обучающую и тестовую выборки и будем хранить перебираемые далее значения гиперпараметров.

In [21]:
# Загрузка и подготовка данных
X = data.iloc[:,:-1].values
y = data.iloc[:, -1].values

X_train_lab = copy(X)
le = LabelEncoder()
for j in range(X.shape[1]): # кодируем значения признаков
    X_train_lab[:, j] = le.fit_transform(X[:, j])
X_train, X_test, y_train, y_test = train_test_split(X_train_lab, y, test_size=0.2, random_state=42)

# Задаем гиперпараметры для всех моделей
lr_params = {'fit_intercept': [True, False]}
gb_params = {'n_estimators': [50, 100, 150], 'max_depth': [3, 5, 7]}
knn_params = {'n_neighbors': [3, 5, 7, 9, 11]}
rf_params = {'n_estimators': [50, 100, 150], 'max_depth': [3, 5, 7]}
svm_params = {'C': [0.001, 0.01, 0.1, 1, 10], 'kernel': ['linear', 'rbf']}
a1_params = {'eps': [0.1, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],\
             'min_samples': [3, 5, 7, 10, 15, 30, 40, 50, 75, 100]\
            }



# Создаем объекты моделей
models = [LinearRegression(),\
          GradientBoostingRegressor(),\
          KNeighborsRegressor(),\
          RandomForestRegressor(),\
          SVR(),\
          A1_Rg()]

params = [lr_params,\
          gb_params,\
          knn_params,\
          rf_params,\
          svm_params,\
          a1_params]

In [22]:
X_train = X_train.astype(int)
X_test = X_test.astype(int)

In [23]:
y_train = y_train.astype(float)
y_test = y_test.astype(float)

In [24]:
X_train.shape

(160, 16)

In [25]:
y_train.shape

(160,)

In [26]:
# Создаем списки метрик и результатов времени работы
results = []
time_results = []

In [27]:
np.sum(X_train == '?')

/tmp/ipykernel_4723/1523399965.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  np.sum(X_train == '?')


0

In [28]:
np.sum(y_train == '?')

/tmp/ipykernel_4723/2693416949.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  np.sum(y_train == '?')


0

Подберем оптимальные значения параметров моделей с помощью GridSearchCV и оценим качество работы моделей с полученными параметрами на кросс-валидации по 5 фолдам: по очередности использовались 4 фолда для обучения и 1 фолд для контроля.

In [29]:
# Используем цикл для обучения и оценки каждой модели
for model, param in tqdm(zip(models, params)):
    start_time = time.time()
    # Используем GridSearchCV для поиска лучших параметров модели
    grid = GridSearchCV(model, param, cv=5, scoring='neg_mean_squared_error')
    # print("____")
    grid.fit(X_train, y_train)
    # Вычисляем среднеквадратичную ошибку на кросс-валидации
    scores = cross_val_score(grid.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    mse = np.abs(np.mean(scores))
    r2 = r2_score(y_test, grid.predict(X_test))    
    # Добавляем результаты в список
    results.append(f"{model.__class__.__name__}:\nBest params:{grid.best_params_}\nMSE: {mse:.2f}\nR2: {r2:.2f}\n")
    # Замеряем время работы
    end_time = time.time()
    time_results.append({'model': model.__class__.__name__, 'time': end_time - start_time})
    
# Выводим результаты
print('Results:')
for res in results:
    print(''.join(res))

print('\nTime results:')
for tres in time_results:
    print(tres)

4it [00:06,  1.84s/it]

       class = 0      
START: Sun Sep 17 18:51:43 2023

FIT TIME: 0.00768
END: Sun Sep 17 18:51:43 2023

       class = 1      
START: Sun Sep 17 18:51:44 2023

FIT TIME: 0.001122
END: Sun Sep 17 18:51:44 2023

       class = 2      
START: Sun Sep 17 18:51:46 2023

FIT TIME: 0.000221
END: Sun Sep 17 18:51:46 2023

       class = 3      
START: Sun Sep 17 18:51:47 2023

FIT TIME: 0.000385
END: Sun Sep 17 18:51:47 2023

       class = 4      
START: Sun Sep 17 18:51:48 2023

FIT TIME: 0.000279
END: Sun Sep 17 18:51:48 2023

       class = 5      
START: Sun Sep 17 18:51:50 2023

FIT TIME: 0.000322
END: Sun Sep 17 18:51:50 2023

       class = 0      
START: Sun Sep 17 18:51:52 2023

FIT TIME: 0.00403
END: Sun Sep 17 18:51:52 2023

       class = 1      
START: Sun Sep 17 18:51:54 2023

FIT TIME: 0.000386
END: Sun Sep 17 18:51:54 2023

       class = 2      
START: Sun Sep 17 18:51:55 2023

FIT TIME: 0.000384
END: Sun Sep 17 18:51:55 2023

       class = 3      
START: Sun Sep 17 18:51:5

/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k


START: Sun Sep 17 18:52:21 2023

       class = 0      


/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k


...

START: Sun Sep 17 19:04:55 2023

       class = 0      


/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k


START: Sun Sep 17 19:04:57 2023

FIT TIME: 0.008797
END: Sun Sep 17 19:04:57 2023

       class = 1      
START: Sun Sep 17 19:04:58 2023

FIT TIME: 0.000118
END: Sun Sep 17 19:04:58 2023

       class = 2      
START: Sun Sep 17 19:04:59 2023

FIT TIME: 0.00034
END: Sun Sep 17 19:04:59 2023

       class = 3      
START: Sun Sep 17 19:05:00 2023

FIT TIME: 0.000487
END: Sun Sep 17 19:05:00 2023

       class = 4      
START: Sun Sep 17 19:05:02 2023

FIT TIME: 0.000632
END: Sun Sep 17 19:05:02 2023

       class = 5      
START: Sun Sep 17 19:05:03 2023

FIT TIME: 0.000563
END: Sun Sep 17 19:05:03 2023

       class = 6      
START: Sun Sep 17 19:05:04 2023

FIT TIME: 0.000756
END: Sun Sep 17 19:05:04 2023

       class = 7      
START: Sun Sep 17 19:05:05 2023

FIT TIME: 0.000271
END: Sun Sep 17 19:05:05 2023

       class = 8      
START: Sun Sep 17 19:05:07 2023

FIT TIME: 0.000548
END: Sun Sep 17 19:05:07 2023

       class = 9      
START: Sun Sep 17 19:05:08 2023

FIT TIME: 0.00

6it [14:47, 147.86s/it]

Results:
LinearRegression:
Best params:{'fit_intercept': False}
MSE: 15985144.51
R2: 0.74

GradientBoostingRegressor:
Best params:{'max_depth': 3, 'n_estimators': 100}
MSE: 8838870.19
R2: 0.94

KNeighborsRegressor:
Best params:{'n_neighbors': 3}
MSE: 16424945.96
R2: 0.73

RandomForestRegressor:
Best params:{'max_depth': 7, 'n_estimators': 150}
MSE: 11151554.69
R2: 0.87

SVR:
Best params:{'C': 10, 'kernel': 'linear'}
MSE: 27074131.91
R2: 0.19

A1_Rg:
Best params:{'eps': 1.0, 'min_samples': 3}
MSE: 26579920.91
R2: 0.29


Time results:
{'model': 'LinearRegression', 'time': 0.04010796546936035}
{'model': 'GradientBoostingRegressor', 'time': 1.5185511112213135}
{'model': 'KNeighborsRegressor', 'time': 0.030768632888793945}
{'model': 'RandomForestRegressor', 'time': 5.146656274795532}
{'model': 'SVR', 'time': 0.09422183036804199}
{'model': 'A1_Rg', 'time': 880.3374741077423}


Оценим качество моделей с оптимальными параметрами на тестовой выборке подсчетом метрик RMSE и R2 score.

In [30]:
model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("LinearRegression:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = GradientBoostingRegressor(max_depth=3, n_estimators=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("GradientBoostingRegressor:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = KNeighborsRegressor(n_neighbors=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("KNeighborsRegressor:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = RandomForestRegressor(max_depth=7, n_estimators=150)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("RandomForestRegressor:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = SVR(C=10, kernel='linear')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("SVR:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = A1_Rg(eps=1.0, min_samples=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("A1_Rg:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

LinearRegression:
Mean Squared Error: 5616.870214600147
R2 Score: 0.7421327084259137
GradientBoostingRegressor:
Mean Squared Error: 2806.8760366764004
R2 Score: 0.9356047332777347
KNeighborsRegressor:
Mean Squared Error: 5758.1189392159295
R2 Score: 0.7290003415476392
RandomForestRegressor:
Mean Squared Error: 4019.2007625064316
R2 Score: 0.8679656654416663
SVR:
Mean Squared Error: 9955.98682628604
R2 Score: 0.1898300936945092
       class = 0      
START: Sun Sep 17 19:07:50 2023

FIT TIME: 0.008465
END: Sun Sep 17 19:07:50 2023

       class = 1      
START: Sun Sep 17 19:07:51 2023

FIT TIME: 0.00031
END: Sun Sep 17 19:07:51 2023

       class = 2      
START: Sun Sep 17 19:07:52 2023

FIT TIME: 0.000731
END: Sun Sep 17 19:07:52 2023

       class = 3      
START: Sun Sep 17 19:07:53 2023

FIT TIME: 0.001049
END: Sun Sep 17 19:07:53 2023

       class = 4      
START: Sun Sep 17 19:07:55 2023

FIT TIME: 0.000611
END: Sun Sep 17 19:07:55 2023

       class = 5      
START: Sun Sep 17

Еще раз протестируем полученные модели и оценим их качество и устойчивость работы, усредняя результаты подсчета метрик и времени работы по 10 независимым запускам на случайных разбиениях данных.

In [31]:
res_rmse = [0.0] * 6
tres = [0.0] * 6
res_r2 = [0.0] * 6
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_train_lab, y, test_size=0.2, random_state=42 + i)
    X_train = X_train.astype(int)
    X_test = X_test.astype(int)
    y_train = y_train.astype(float)
    y_test = y_test.astype(float)
    
    start = time.time()
    model = LinearRegression(fit_intercept=False)
    model.fit(X_train, y_train)
    res_rmse[0] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[0] += r2_score(y_test, model.predict(X_test))
    tres[0] += time.time() - start
    
    start = time.time()
    model = GradientBoostingRegressor(max_depth=3, n_estimators=100)
    model.fit(X_train, y_train)
    res_rmse[1] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[1] += r2_score(y_test, model.predict(X_test))
    tres[1] += time.time() - start
    
    start = time.time()
    model = KNeighborsRegressor(n_neighbors=3)
    model.fit(X_train, y_train)
    res_rmse[2] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[2] += r2_score(y_test, model.predict(X_test))
    tres[2] += time.time() - start
    
    start = time.time()
    model = RandomForestRegressor(max_depth=7, n_estimators=100)
    model.fit(X_train, y_train)
    res_rmse[3] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[3] += r2_score(y_test, model.predict(X_test))
    tres[3] += time.time() - start
    
    start = time.time()
    model = SVR(C=10, kernel='linear')
    model.fit(X_train, y_train)
    res_rmse[4] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[4] += r2_score(y_test, model.predict(X_test))
    tres[4] += time.time() - start
    
    start = time.time()
    model = A1_Rg(eps=1.0, min_samples=5)
    model.fit(X_train, y_train)
    res_rmse[5] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[5] += r2_score(y_test, model.predict(X_test))
    tres[5] += time.time() - start
    
print("LinearRegression:")
print("Mean Squared Error:", res_rmse[0] / 10)
print("R2 Score:", res_r2[0] / 10)
print("Time:", tres[0] / 10)

print("GradientBoostingRegressor:")
print("Mean Squared Error:", res_rmse[1] / 10)
print("R2 Score:", res_r2[1] / 10)
print("Time:", tres[1] / 10)

print("KNeighborsRegressor:")
print("Mean Squared Error:", res_rmse[2] / 10)
print("R2 Score:", res_r2[2] / 10)
print("Time:", tres[2] / 10)

print("RandomForestRegressor:")
print("Mean Squared Error:", res_rmse[3] / 10)
print("R2 Score:", res_r2[3] / 10)
print("Time:", tres[3] / 10)

print("SVR:")
print("Mean Squared Error:", res_rmse[4] / 10)
print("R2 Score:", res_r2[4] / 10)
print("Time:", tres[4] / 10)

print("A1_Rg:")
print("Mean Squared Error:", res_rmse[5] / 10)
print("R2 Score:", res_r2[5] / 10)
print("Time:", tres[5] / 10)

       class = 0      
START: Sun Sep 17 19:08:47 2023

FIT TIME: 0.00402
END: Sun Sep 17 19:08:47 2023

       class = 1      
START: Sun Sep 17 19:08:48 2023

FIT TIME: 0.000902
END: Sun Sep 17 19:08:48 2023

       class = 0      
START: Sun Sep 17 19:08:50 2023

FIT TIME: 0.003851
END: Sun Sep 17 19:08:50 2023

       class = 1      
START: Sun Sep 17 19:08:52 2023

FIT TIME: 0.001618
END: Sun Sep 17 19:08:52 2023

       class = 2      
START: Sun Sep 17 19:08:53 2023

FIT TIME: 0.001912
END: Sun Sep 17 19:08:53 2023

       class = 3      
START: Sun Sep 17 19:08:54 2023

FIT TIME: 0.000375
END: Sun Sep 17 19:08:54 2023

       class = 0      
START: Sun Sep 17 19:08:56 2023

FIT TIME: 0.001937
END: Sun Sep 17 19:08:56 2023

       class = 1      
START: Sun Sep 17 19:08:57 2023

FIT TIME: 0.001551
END: Sun Sep 17 19:08:57 2023

       class = 0      
START: Sun Sep 17 19:08:59 2023

FIT TIME: 0.006428
END: Sun Sep 17 19:08:59 2023

       class = 1      
START: Sun Sep 17 19:09:

/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k
/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k


START: Sun Sep 17 19:09:15 2023



/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k
/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k


       class = 0      
START: Sun Sep 17 19:09:16 2023

FIT TIME: 0.00352
END: Sun Sep 17 19:09:16 2023

       class = 1      
START: Sun Sep 17 19:09:18 2023

FIT TIME: 0.00139
END: Sun Sep 17 19:09:18 2023

       class = 2      
START: Sun Sep 17 19:09:19 2023

FIT TIME: 0.001567
END: Sun Sep 17 19:09:19 2023

LinearRegression:
Mean Squared Error: 4221.198662294704
R2 Score: 0.7274577656827154
Time: 0.002140188217163086
GradientBoostingRegressor:
Mean Squared Error: 2481.6261124778343
R2 Score: 0.8985895480610061
Time: 0.03172686100006104
KNeighborsRegressor:
Mean Squared Error: 4291.68912679671
R2 Score: 0.7066220829369387
Time: 0.0015622377395629883
RandomForestRegressor:
Mean Squared Error: 2750.414147211072
R2 Score: 0.8804116745989352
Time: 0.10689783096313477
SVR:
Mean Squared Error: 6601.695411394847
R2 Score: 0.35204765070751015
Time: 0.001992154121398926
A1_Rg:
Mean Squared Error: 6522.773033739815
R2 Score: 0.36136576161613426
Time: 3.2565602540969847


In [32]:
results

["LinearRegression:\nBest params:{'fit_intercept': False}\nMSE: 15985144.51\nR2: 0.74\n",
 "GradientBoostingRegressor:\nBest params:{'max_depth': 3, 'n_estimators': 100}\nMSE: 8838870.19\nR2: 0.94\n",
 "KNeighborsRegressor:\nBest params:{'n_neighbors': 3}\nMSE: 16424945.96\nR2: 0.73\n",
 "RandomForestRegressor:\nBest params:{'max_depth': 7, 'n_estimators': 150}\nMSE: 11151554.69\nR2: 0.87\n",
 "SVR:\nBest params:{'C': 10, 'kernel': 'linear'}\nMSE: 27074131.91\nR2: 0.19\n",
 "A1_Rg:\nBest params:{'eps': 1.0, 'min_samples': 3}\nMSE: 26579920.91\nR2: 0.29\n"]

In [33]:
time_results

[{'model': 'LinearRegression', 'time': 0.04010796546936035},
 {'model': 'GradientBoostingRegressor', 'time': 1.5185511112213135},
 {'model': 'KNeighborsRegressor', 'time': 0.030768632888793945},
 {'model': 'RandomForestRegressor', 'time': 5.146656274795532},
 {'model': 'SVR', 'time': 0.09422183036804199},
 {'model': 'A1_Rg', 'time': 880.3374741077423}]

## Computer hardware

Датасет "Computer Hardware" из ресурса UCI содержит информацию о характеристиках компьютерного оборудования.

Число прецедентов (наблюдений) в датасете составляет 209. Каждое наблюдение представляет собой информацию о различных признаках компьютера.

Датасет содержит следующие признаки (характеристики) и их значения:

1. Vendor Name (название производителя компьютера)
   - Принимаемые значения: 'adviser', 'amdahl', 'apollo', 'basf', 'bti', 'burroughs', 'c.r.d', 'cambex', 'cdc', 'dec', 'dg', 'formation', 'four-phase', 'gould', 'harris', 'honeywell', 'hp', 'ibm', 'ipl', 'magnuson', 'microdata', 'nas', 'ncr', 'nixdorf', 'perkin-elmer', 'prime', 'siemens', 'sperry', 'sratus', 'wang'
   
2. Model Name (название модели компьютера)
   - Принимаемые значения: Текстовые значения, уникальные для каждой модели компьютера.
   
3. Machine Cycle Time (MYCT) (максимальное число циклов работы, миллионы тактов за одну минуту)
   - Диапазон значений: 17, 1500
   
4. Minimum Main Memory (MMIN) (минимальное число памяти в килобайтах)
   - Диапазон значений: 64, 32000
   
5. Maximum Main Memory (MMAX) (максимальное число памяти в килобайтах)
   - Диапазон значений: 64, 64000

6. Cache Memory (CACH) (размер кэша памяти в килобайтах)
   - Диапазон значений: 0, 256

7. Minimum Number of Channels (CHMIN) (минимальное число каналов ввода-вывода)
   - Диапазон значений: 0, 52

8. Maximum Number of Channels (CHMAX) (максимальное число каналов ввода-вывода)
   - Диапазон значений: 0, 176

9. Published Relative Performance (PRP) (рейтинг производительности компьютера)
   - Диапазон значений: 6, 1500

10. Estimated Relative Performance (ERP) (средняя эффективность работы компьютера)
    - Диапазон значений: 15, 1238

In [34]:
data = pd.read_csv('machine.data', header=None)
data

,0,1,2,3,4,5,6,7,8,9
0,adviser,32/60,125,256,6000,256,16,128,198,199
1,amdahl,470v/7,29,8000,32000,32,8,32,269,253
2,amdahl,470v/7a,29,8000,32000,32,8,32,220,253
3,amdahl,470v/7b,29,8000,32000,32,8,32,172,253
4,amdahl,470v/7c,29,8000,16000,32,8,16,132,132
...,...,...,...,...,...,...,...,...,...,...
204,sperry,80/8,124,1000,8000,0,1,8,42,37
205,sperry,90/80-model-3,98,1000,8000,32,2,8,46,50
206,sratus,32,125,2000,8000,0,2,14,52,41
207,wang,vs-100,480,512,8000,32,0,0,67,47


Уберем столбец c последним признаком, так как будем предсказывать рейтинг производительности компьютера (PRP).

In [35]:
data.drop(9, axis=1, inplace=True)

In [36]:
data

,0,1,2,3,4,5,6,7,8
0,adviser,32/60,125,256,6000,256,16,128,198
1,amdahl,470v/7,29,8000,32000,32,8,32,269
2,amdahl,470v/7a,29,8000,32000,32,8,32,220
3,amdahl,470v/7b,29,8000,32000,32,8,32,172
4,amdahl,470v/7c,29,8000,16000,32,8,16,132
...,...,...,...,...,...,...,...,...,...
204,sperry,80/8,124,1000,8000,0,1,8,42
205,sperry,90/80-model-3,98,1000,8000,32,2,8,46
206,sratus,32,125,2000,8000,0,2,14,52
207,wang,vs-100,480,512,8000,32,0,0,67


In [37]:
for i, col in enumerate(data.columns[:-1]):
    print(data[col].dtype)

object
object
int64
int64
int64
int64
int64
int64


Для кодирования вещественных признаков применяется процедура деления на интервалы. Выбирается 5 порогов для признака col, делящих обучающую выборку по этому признаку на 5 равных частей, после чего вещественные значения заменяются на категориальные интервалы.

In [38]:
for i, col in enumerate(data.columns[2:-1]):
    min_v = min(data[col])
    max_v = max(data[col])
    length = (max_v - min_v) / 5
    data[col] = data[col].apply(lambda x: round((x - min_v) / length))

In [39]:
for i, col in enumerate(data.columns[:-1]):
    print(i, np.unique(data[col]).shape[0])

0 30
1 209
2 6
3 4
4 4
5 5
6 5
7 6


Закодируем категориальные признаки в числовые значения с помощью LabelEncoder, разделим данные на обучающую и тестовую выборки и будем хранить перебираемые далее значения гиперпараметров.

In [45]:
# Загрузка и подготовка данных
X = data.iloc[:,:-1].values
y = data.iloc[:, -1].values

X_train_lab = copy(X)
le = LabelEncoder()
for j in range(X.shape[1]): # кодируем значения признаков
    X_train_lab[:, j] = le.fit_transform(X[:, j])
X_train, X_test, y_train, y_test = train_test_split(X_train_lab, y, test_size=0.2, random_state=42)

lr_params = {'fit_intercept': [True, False]}
gb_params = {'n_estimators': [50, 100, 150], 'max_depth': [3, 5, 7]}
knn_params = {'n_neighbors': [3, 5, 7, 9, 11]}
rf_params = {'n_estimators': [50, 100, 150], 'max_depth': [3, 5, 7]}
svm_params = {'C': [0.001, 0.01, 0.1, 1, 10], 'kernel': ['linear', 'rbf']}
a1_params = {'eps': [0.1, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],\
             'min_samples': [3, 5, 7, 10, 15, 30, 40, 50, 75, 100]\
            }



# Создаем объекты моделей и параметров восстановления
models = [LinearRegression(),\
          GradientBoostingRegressor(),\
          KNeighborsRegressor(),\
          RandomForestRegressor(),\
          SVR(),\
          A1_Rg()]

params = [lr_params,\
          gb_params,\
          knn_params,\
          rf_params,\
          svm_params,\
          a1_params]

In [46]:
X_train = X_train.astype(int)
X_test = X_test.astype(int)

In [47]:
y_train = y_train.astype(float)
y_test = y_test.astype(float)

In [48]:
X_train.shape

(167, 8)

In [49]:
y_train.shape

(167,)

In [50]:
# Создаем списки результатов и метрик времени работы
results = []
time_results = []

Подберем оптимальные значения параметров моделей с помощью GridSearchCV и оценим качество работы моделей с полученными параметрами на кросс-валидации по 5 фолдам: по очередности использовались 4 фолда для обучения и 1 фолд для контроля.


In [51]:
# Используем цикл для обучения и оценки каждой модели
for model, param in tqdm(zip(models, params)):
    start_time = time.time()
    # Используем GridSearchCV для поиска лучших параметров модели
    grid = GridSearchCV(model, param, cv=5, scoring='neg_mean_squared_error')
    # print("____")
    grid.fit(X_train, y_train)
    # Вычисляем среднеквадратичную ошибку на кросс-валидации
    # print("*****")
    scores = cross_val_score(grid.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    mse = np.abs(np.mean(scores))
    r2 = r2_score(y_test, grid.predict(X_test))    
    # Добавляем результаты в список
    results.append(f"{model.__class__.__name__}:\nBest params:{grid.best_params_}\nMSE: {mse:.2f}\nR2: {r2:.2f}\n")
    # Замеряем время работы
    end_time = time.time()
    time_results.append({'model': model.__class__.__name__, 'time': end_time - start_time})
    
# Выводим результаты
print('Results:')
for res in results:
    print(''.join(res))

print('\nTime results:')
for tres in time_results:
    print(tres)

5it [00:07,  1.49s/it]

       class = 0      
START: Sun Sep 17 19:13:16 2023

       class = 0      


/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k


START: Sun Sep 17 19:13:17 2023

       class = 0      


/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k


...

START: Sun Sep 17 19:22:37 2023

       class = 0      


/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k


START: Sun Sep 17 19:22:38 2023

FIT TIME: 0.00249
END: Sun Sep 17 19:22:38 2023

       class = 1      
START: Sun Sep 17 19:22:40 2023

FIT TIME: 0.000227
END: Sun Sep 17 19:22:40 2023

       class = 2      
START: Sun Sep 17 19:22:42 2023

FIT TIME: 0.000207
END: Sun Sep 17 19:22:42 2023

       class = 3      
START: Sun Sep 17 19:22:44 2023

FIT TIME: 0.000217
END: Sun Sep 17 19:22:44 2023

       class = 4      
START: Sun Sep 17 19:22:45 2023

FIT TIME: 0.000205
END: Sun Sep 17 19:22:45 2023

       class = 5      
START: Sun Sep 17 19:22:47 2023

FIT TIME: 0.000206
END: Sun Sep 17 19:22:47 2023

       class = 6      
START: Sun Sep 17 19:22:49 2023

FIT TIME: 0.000175
END: Sun Sep 17 19:22:49 2023

       class = 7      
START: Sun Sep 17 19:22:51 2023

FIT TIME: 0.000276
END: Sun Sep 17 19:22:51 2023

       class = 8      
START: Sun Sep 17 19:22:53 2023

FIT TIME: 0.000171
END: Sun Sep 17 19:22:53 2023

       class = 0      
START: Sun Sep 17 19:22:54 2023

FIT TIME: 0.00

6it [10:23, 103.88s/it]

Results:
LinearRegression:
Best params:{'fit_intercept': False}
MSE: 8069.49
R2: 0.76

GradientBoostingRegressor:
Best params:{'max_depth': 3, 'n_estimators': 100}
MSE: 7296.90
R2: 0.86

KNeighborsRegressor:
Best params:{'n_neighbors': 5}
MSE: 16179.03
R2: 0.47

RandomForestRegressor:
Best params:{'max_depth': 5, 'n_estimators': 150}
MSE: 7457.89
R2: 0.89

SVR:
Best params:{'C': 10, 'kernel': 'linear'}
MSE: 8009.72
R2: 0.57

A1_Rg:
Best params:{'eps': 1.0, 'min_samples': 3}
MSE: 15969.37
R2: 0.10


Time results:
{'model': 'LinearRegression', 'time': 0.028725385665893555}
{'model': 'GradientBoostingRegressor', 'time': 1.454827070236206}
{'model': 'KNeighborsRegressor', 'time': 0.061226844787597656}
{'model': 'RandomForestRegressor', 'time': 4.322224855422974}
{'model': 'SVR', 'time': 1.2252204418182373}
{'model': 'A1_Rg', 'time': 616.2130901813507}


Оценим качество моделей с оптимальными параметрами на тестовой выборке подсчетом метрик RMSE и R2 score.


In [52]:
model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("LinearRegression:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = GradientBoostingRegressor(max_depth=3, n_estimators=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("GradientBoostingRegressor:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = KNeighborsRegressor(n_neighbors=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("KNeighborsRegressor:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = RandomForestRegressor(max_depth=5, n_estimators=150)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("RandomForestRegressor:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = SVR(C=10, kernel='linear')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("SVR:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = A1_Rg(eps=1.0, min_samples=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("A1_Rg:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

LinearRegression:
Mean Squared Error: 110.96370173181926
R2 Score: 0.7581279860336971
GradientBoostingRegressor:
Mean Squared Error: 80.21375795888412
R2 Score: 0.873607450379659
KNeighborsRegressor:
Mean Squared Error: 164.92180253799359
R2 Score: 0.4657064961550147
RandomForestRegressor:
Mean Squared Error: 79.23927623030669
R2 Score: 0.8766597716034961
SVR:
Mean Squared Error: 148.1963221842077
R2 Score: 0.5685816635807663
       class = 0      
START: Sun Sep 17 19:25:04 2023

FIT TIME: 0.001418
END: Sun Sep 17 19:25:04 2023

       class = 1      
START: Sun Sep 17 19:25:06 2023

FIT TIME: 0.000141
END: Sun Sep 17 19:25:06 2023

       class = 2      
START: Sun Sep 17 19:25:08 2023

FIT TIME: 7.1e-05
END: Sun Sep 17 19:25:08 2023

       class = 3      
START: Sun Sep 17 19:25:09 2023

FIT TIME: 0.000241
END: Sun Sep 17 19:25:09 2023

       class = 4      
START: Sun Sep 17 19:25:11 2023

FIT TIME: 0.000158
END: Sun Sep 17 19:25:11 2023

       class = 5      
START: Sun Sep 17 

Еще раз протестируем полученные модели и оценим их качество и устойчивость работы, усредняя результаты подсчета метрик и времени работы по 10 независимым запускам на случайных разбиениях данных.

In [53]:
res_rmse = [0.0] * 6
tres = [0.0] * 6
res_r2 = [0.0] * 6
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_train_lab, y, test_size=0.2, random_state=42 + i)
    X_train = X_train.astype(int)
    X_test = X_test.astype(int)
    y_train = y_train.astype(float)
    y_test = y_test.astype(float)
    
    start = time.time()
    model = LinearRegression(fit_intercept=False)
    model.fit(X_train, y_train)
    res_rmse[0] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[0] += r2_score(y_test, model.predict(X_test))
    tres[0] += time.time() - start
    
    start = time.time()
    model = GradientBoostingRegressor(max_depth=3, n_estimators=100)
    model.fit(X_train, y_train)
    res_rmse[1] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[1] += r2_score(y_test, model.predict(X_test))
    tres[1] += time.time() - start
    
    start = time.time()
    model = KNeighborsRegressor(n_neighbors=5)
    model.fit(X_train, y_train)
    res_rmse[2] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[2] += r2_score(y_test, model.predict(X_test))
    tres[2] += time.time() - start
    
    start = time.time()
    model = RandomForestRegressor(max_depth=5, n_estimators=150)
    model.fit(X_train, y_train)
    res_rmse[3] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[3] += r2_score(y_test, model.predict(X_test))
    tres[3] += time.time() - start
    
    start = time.time()
    model = SVR(C=10, kernel='linear')
    model.fit(X_train, y_train)
    res_rmse[4] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[4] += r2_score(y_test, model.predict(X_test))
    tres[4] += time.time() - start
    
    start = time.time()
    model = A1_Rg(eps=1.0, min_samples=3)
    model.fit(X_train, y_train)
    res_rmse[5] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[5] += r2_score(y_test, model.predict(X_test))
    tres[5] += time.time() - start
    
print("LinearRegression:")
print("Mean Squared Error:", res_rmse[0] / 10)
print("R2 Score:", res_r2[0] / 10)
print("Time:", tres[0] / 10)

print("GradientBoostingRegressor:")
print("Mean Squared Error:", res_rmse[1] / 10)
print("R2 Score:", res_r2[1] / 10)
print("Time:", tres[1] / 10)

print("KNeighborsRegressor:")
print("Mean Squared Error:", res_rmse[2] / 10)
print("R2 Score:", res_r2[2] / 10)
print("Time:", tres[2] / 10)

print("RandomForestRegressor:")
print("Mean Squared Error:", res_rmse[3] / 10)
print("R2 Score:", res_r2[3] / 10)
print("Time:", tres[3] / 10)

print("SVR:")
print("Mean Squared Error:", res_rmse[4] / 10)
print("R2 Score:", res_r2[4] / 10)
print("Time:", tres[4] / 10)

print("A1_Rg:")
print("Mean Squared Error:", res_rmse[5] / 10)
print("R2 Score:", res_r2[5] / 10)
print("Time:", tres[5] / 10)

       class = 0      
START: Sun Sep 17 19:26:21 2023

FIT TIME: 0.001353
END: Sun Sep 17 19:26:21 2023

       class = 1      
START: Sun Sep 17 19:26:23 2023

FIT TIME: 8.2e-05
END: Sun Sep 17 19:26:23 2023

       class = 2      
START: Sun Sep 17 19:26:25 2023

FIT TIME: 0.000144
END: Sun Sep 17 19:26:25 2023

       class = 3      
START: Sun Sep 17 19:26:27 2023

FIT TIME: 0.0002
END: Sun Sep 17 19:26:27 2023

       class = 4      
START: Sun Sep 17 19:26:29 2023

FIT TIME: 0.000194
END: Sun Sep 17 19:26:29 2023

       class = 5      
START: Sun Sep 17 19:26:31 2023

FIT TIME: 0.000135
END: Sun Sep 17 19:26:31 2023

       class = 6      
START: Sun Sep 17 19:26:33 2023

FIT TIME: 0.000213
END: Sun Sep 17 19:26:33 2023

       class = 7      
START: Sun Sep 17 19:26:35 2023

FIT TIME: 0.000147
END: Sun Sep 17 19:26:35 2023

       class = 8      
START: Sun Sep 17 19:26:37 2023

FIT TIME: 0.000169
END: Sun Sep 17 19:26:37 2023

       class = 0      
START: Sun Sep 17 19:26:39

## Servo

Датасет "Servo" содержит информацию о характеристиках серводвигателя для предсказания времени реакции.

Размерность решаемой задачи: 167 (объектов/наблюдений) * 6 (признаков)

Датасет содержит следующие признаки:

1. Motor (тип мотора)
   - Принимаемые значения: 'A', 'B', 'C', 'D', 'E'

2. Screw (тип винта)
   - Принимаемые значения: 'A', 'B', 'C', 'D', E'

3. Pgain (пропорциональный коэффициент усиления)
   - Диапазон значений: 3, 6

4. Vgain (коэффициент инегрирующего действия)
   - Диапазон значений: 1, 5
   
5. Output (время реакции)
   - Принимаемые значения: [0.13, 7.10]

Значения каждого признака представляют различные характеристики серводвигателя. Признаки Motor, Screw, Pgain, Vgain являются входными переменными, а признак Output - целевой переменной для предсказания времени реакции. Категориальные признаки Motor и Screw принимают значения из заданных списков. Всего в датасете имеется 3 числовых признака и 2 категориальных признака.

In [93]:
data = pd.read_csv('servo.data', header=None)
data

,0,1,2,3,4
0,E,E,5,4,0.281251
1,B,D,6,5,0.506252
2,D,D,4,3,0.356251
3,B,A,3,2,5.500033
4,D,B,6,5,0.356251
...,...,...,...,...,...
162,B,C,3,2,4.499986
163,B,E,3,1,3.699967
164,C,D,4,3,0.956256
165,A,B,3,2,4.499986


In [94]:
for i, col in enumerate(data.columns[:-1]):
    print(data[col].dtype)

object
object
int64
int64


In [96]:
data

,0,1,2,3,4
0,E,E,5,4,0.281251
1,B,D,6,5,0.506252
2,D,D,4,3,0.356251
3,B,A,3,2,5.500033
4,D,B,6,5,0.356251
...,...,...,...,...,...
162,B,C,3,2,4.499986
163,B,E,3,1,3.699967
164,C,D,4,3,0.956256
165,A,B,3,2,4.499986


Закодируем категориальные признаки в числовые значения с помощью LabelEncoder, разделим данные на обучающую и тестовую выборки и будем хранить перебираемые 
далее значения гиперпараметров.


In [95]:
# Загрузка и подготовка данных
# data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data', 
                   # delim_whitespace=True, header=None)
X = data.iloc[:,:-1].values
y = data.iloc[:, -1].values

X_train_lab = copy(X)
le = LabelEncoder()
for j in range(X.shape[1]): # кодируем значения признаков
    X_train_lab[:, j] = le.fit_transform(X[:, j])
X_train, X_test, y_train, y_test = train_test_split(X_train_lab, y, test_size=0.2, random_state=42)


# # Scale the features
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# Задаем гиперпараметры для всех моделей
# lr_params = {'C': [0.001, 0.01, 0.05, 0.1, 0.15, 0.5, 1, 10]}
# gb_params = {'n_estimators': [50, 100, 150, 300, 500, 750, 1000, 1500, 2000], 'max_depth': [3, 5, 7, 10, 15]}
# knn_params = {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance']}
# rf_params = {'n_estimators': [14, 30, 50, 75, 100, 125, 150, 200, 300, 500], 'max_depth': [3, 5, 7, 10, 15]}
# svm_params = {'C': [0.001, 0.01, 0.05, 0.1, 0.15, 0.5, 1, 10], 'kernel': ['linear', 'rbf']}

lr_params = {'fit_intercept': [True, False]}
gb_params = {'n_estimators': [50, 100, 150], 'max_depth': [3, 5, 7]}
knn_params = {'n_neighbors': [3, 5, 7, 9, 11]}
rf_params = {'n_estimators': [50, 100, 150], 'max_depth': [3, 5, 7]}
svm_params = {'C': [0.001, 0.01, 0.1, 1, 10], 'kernel': ['linear', 'rbf']}
a1_params = {'eps': [0.1, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],\
             'min_samples': [3, 5, 7, 10, 15, 30, 40, 50, 75, 100]\
            }



# Создаем объекты моделей и параметров восстановления
models = [LinearRegression(),\
          GradientBoostingRegressor(),\
          KNeighborsRegressor(),\
          RandomForestRegressor(),\
          SVR(),\
          A1_Rg()]

params = [lr_params,\
          gb_params,\
          knn_params,\
          rf_params,\
          svm_params,\
          a1_params]

In [97]:
X_train = X_train.astype(int)
X_test = X_test.astype(int)

In [98]:
y_train = y_train.astype(float)
y_test = y_test.astype(float)

In [99]:
X_train.shape

(133, 4)

In [100]:
y_train.shape

(133,)

In [101]:
# Создаем списки результатов и метрик времени работы
results = []
time_results = []

In [102]:
np.sum(X_train == '?')

/tmp/ipykernel_4723/1523399965.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  np.sum(X_train == '?')


0

In [103]:
np.sum(y_train == '?')

/tmp/ipykernel_4723/2693416949.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  np.sum(y_train == '?')


0

Подберем оптимальные значения параметров моделей с помощью GridSearchCV и оценим качество работы моделей с полученными параметрами на кросс-валидации по 5 фолдам: по очередности использовались 4 фолда для обучения и 1 фолд для контроля.



In [104]:
# Используем цикл для обучения и оценки каждой модели
for model, param in tqdm(zip(models, params)):
    start_time = time.time()
    # Используем GridSearchCV для поиска лучших параметров модели
    grid = GridSearchCV(model, param, cv=5, scoring='neg_mean_squared_error')
    # print("____")
    grid.fit(X_train, y_train)
    # Вычисляем среднеквадратичную ошибку на кросс-валидации
    # print("*****")
    scores = cross_val_score(grid.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    mse = np.abs(np.mean(scores))
    r2 = r2_score(y_test, grid.predict(X_test))    
    # Добавляем результаты в список
    results.append(f"{model.__class__.__name__}:\nBest params:{grid.best_params_}\nMSE: {mse:.2f}\nR2: {r2:.2f}\n")
    # Замеряем время работы
    end_time = time.time()
    time_results.append({'model': model.__class__.__name__, 'time': end_time - start_time})
    
# Выводим результаты
print('Results:')
for res in results:
    print(''.join(res))

print('\nTime results:')
for tres in time_results:
    print(tres)

5it [00:04,  1.20it/s]

       class = 0      
START: Sun Sep 17 21:10:55 2023

       class = 0      


/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k


START: Sun Sep 17 21:10:56 2023

       class = 0      


/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k


...

START: Sun Sep 17 21:22:10 2023

       class = 0      


/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k


START: Sun Sep 17 21:22:12 2023

FIT TIME: 6.3e-05
END: Sun Sep 17 21:22:12 2023

       class = 1      
START: Sun Sep 17 21:22:14 2023

FIT TIME: 0.001026
END: Sun Sep 17 21:22:14 2023

       class = 2      
START: Sun Sep 17 21:22:15 2023

FIT TIME: 0.000411
END: Sun Sep 17 21:22:15 2023

       class = 0      
START: Sun Sep 17 21:22:17 2023

FIT TIME: 0.000204
END: Sun Sep 17 21:22:17 2023

       class = 1      
START: Sun Sep 17 21:22:19 2023

FIT TIME: 0.000475
END: Sun Sep 17 21:22:19 2023

       class = 2      
START: Sun Sep 17 21:22:20 2023

FIT TIME: 0.000221
END: Sun Sep 17 21:22:20 2023

       class = 3      
START: Sun Sep 17 21:22:22 2023

FIT TIME: 0.00015
END: Sun Sep 17 21:22:22 2023

       class = 0      
START: Sun Sep 17 21:22:23 2023

FIT TIME: 0.000118
END: Sun Sep 17 21:22:23 2023

       class = 1      
START: Sun Sep 17 21:22:25 2023

FIT TIME: 0.000744
END: Sun Sep 17 21:22:25 2023

       class = 2      
START: Sun Sep 17 21:22:27 2023

FIT TIME: 0.000

6it [12:03, 120.55s/it]

Results:
LinearRegression:
Best params:{'fit_intercept': True}
MSE: 1.46
R2: 0.59

GradientBoostingRegressor:
Best params:{'max_depth': 5, 'n_estimators': 50}
MSE: 0.22
R2: 0.87

KNeighborsRegressor:
Best params:{'n_neighbors': 3}
MSE: 1.11
R2: 0.56

RandomForestRegressor:
Best params:{'max_depth': 7, 'n_estimators': 50}
MSE: 0.28
R2: 0.82

SVR:
Best params:{'C': 10, 'kernel': 'rbf'}
MSE: 0.64
R2: 0.96

A1_Rg:
Best params:{'eps': 1.0, 'min_samples': 3}
MSE: 1.88
R2: 0.28


Time results:
{'model': 'LinearRegression', 'time': 0.034688472747802734}
{'model': 'GradientBoostingRegressor', 'time': 0.8962743282318115}
{'model': 'KNeighborsRegressor', 'time': 0.032668352127075195}
{'model': 'RandomForestRegressor', 'time': 3.20176362991333}
{'model': 'SVR', 'time': 0.1007390022277832}
{'model': 'A1_Rg', 'time': 719.0516612529755}


Оценим качество моделей с оптимальными параметрами на тестовой выборке подсчетом метрик RMSE и R2 score.


In [105]:
model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("LinearRegression:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = GradientBoostingRegressor(max_depth=5, n_estimators=50)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("GradientBoostingRegressor:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = KNeighborsRegressor(n_neighbors=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("KNeighborsRegressor:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = RandomForestRegressor(max_depth=7, n_estimators=50)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("RandomForestRegressor:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = SVR(C=10, kernel='rbf')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("SVR:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = A1_Rg(eps=1.0, min_samples=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("A1_Rg:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

LinearRegression:
Mean Squared Error: 0.9925995219987878
R2 Score: 0.5926992250439136
GradientBoostingRegressor:
Mean Squared Error: 0.5707742346479715
R2 Score: 0.8653222557392308
KNeighborsRegressor:
Mean Squared Error: 1.026531987269275
R2 Score: 0.5643757111337067
RandomForestRegressor:
Mean Squared Error: 0.5921951163315385
R2 Score: 0.8550237853452385
SVR:
Mean Squared Error: 0.31153015879773305
R2 Score: 0.9598794104695438
       class = 0      
START: Sun Sep 17 21:24:09 2023

FIT TIME: 0.000297
END: Sun Sep 17 21:24:09 2023

       class = 1      
START: Sun Sep 17 21:24:11 2023

FIT TIME: 0.000982
END: Sun Sep 17 21:24:11 2023

       class = 2      
START: Sun Sep 17 21:24:13 2023

FIT TIME: 0.000372
END: Sun Sep 17 21:24:13 2023

A1_Rg:
Mean Squared Error: 1.334227398719258
R2 Score: 0.26408641095023044


Еще раз протестируем полученные модели и оценим их качество и устойчивость работы, усредняя результаты подсчета метрик и времени работы по 10 независимым запускам на случайных разбиениях данных.

In [106]:
res_rmse = [0.0] * 6
tres = [0.0] * 6
res_r2 = [0.0] * 6
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_train_lab, y, test_size=0.2, random_state=42 + i)
    X_train = X_train.astype(int)
    X_test = X_test.astype(int)
    y_train = y_train.astype(float)
    y_test = y_test.astype(float)
    
    start = time.time()
    model = LinearRegression(fit_intercept=True)
    model.fit(X_train, y_train)
    res_rmse[0] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[0] += r2_score(y_test, model.predict(X_test))
    tres[0] += time.time() - start
    
    start = time.time()
    model = GradientBoostingRegressor(max_depth=5, n_estimators=50)
    model.fit(X_train, y_train)
    res_rmse[1] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[1] += r2_score(y_test, model.predict(X_test))
    tres[1] += time.time() - start
    
    start = time.time()
    model = KNeighborsRegressor(n_neighbors=3)
    model.fit(X_train, y_train)
    res_rmse[2] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[2] += r2_score(y_test, model.predict(X_test))
    tres[2] += time.time() - start
    
    start = time.time()
    model = RandomForestRegressor(max_depth=7, n_estimators=50)
    model.fit(X_train, y_train)
    res_rmse[3] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[3] += r2_score(y_test, model.predict(X_test))
    tres[3] += time.time() - start
    
    start = time.time()
    model = SVR(C=10, kernel='rbf')
    model.fit(X_train, y_train)
    res_rmse[4] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[4] += r2_score(y_test, model.predict(X_test))
    tres[4] += time.time() - start
    
    start = time.time()
    model = A1_Rg(eps=1.0, min_samples=3)
    model.fit(X_train, y_train)
    res_rmse[5] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[5] += r2_score(y_test, model.predict(X_test))
    tres[5] += time.time() - start
    
print("LinearRegression:")
print("Mean Squared Error:", res_rmse[0] / 10)
print("R2 Score:", res_r2[0] / 10)
print("Time:", tres[0] / 10)

print("GradientBoostingRegressor:")
print("Mean Squared Error:", res_rmse[1] / 10)
print("R2 Score:", res_r2[1] / 10)
print("Time:", tres[1] / 10)

print("KNeighborsRegressor:")
print("Mean Squared Error:", res_rmse[2] / 10)
print("R2 Score:", res_r2[2] / 10)
print("Time:", tres[2] / 10)

print("RandomForestRegressor:")
print("Mean Squared Error:", res_rmse[3] / 10)
print("R2 Score:", res_r2[3] / 10)
print("Time:", tres[3] / 10)

print("SVR:")
print("Mean Squared Error:", res_rmse[4] / 10)
print("R2 Score:", res_r2[4] / 10)
print("Time:", tres[4] / 10)

print("A1_Rg:")
print("Mean Squared Error:", res_rmse[5] / 10)
print("R2 Score:", res_r2[5] / 10)
print("Time:", tres[5] / 10)

       class = 0      
START: Sun Sep 17 21:25:12 2023

FIT TIME: 0.000151
END: Sun Sep 17 21:25:12 2023

       class = 1      
START: Sun Sep 17 21:25:14 2023

FIT TIME: 0.000462
END: Sun Sep 17 21:25:14 2023

       class = 2      
START: Sun Sep 17 21:25:16 2023

FIT TIME: 0.000377
END: Sun Sep 17 21:25:16 2023

       class = 0      
START: Sun Sep 17 21:25:18 2023

FIT TIME: 0.000139
END: Sun Sep 17 21:25:18 2023

       class = 1      
START: Sun Sep 17 21:25:20 2023

FIT TIME: 0.000358
END: Sun Sep 17 21:25:20 2023

       class = 2      
START: Sun Sep 17 21:25:21 2023

FIT TIME: 0.000698
END: Sun Sep 17 21:25:21 2023

       class = 3      
START: Sun Sep 17 21:25:23 2023

FIT TIME: 0.00042
END: Sun Sep 17 21:25:23 2023

       class = 0      
START: Sun Sep 17 21:25:25 2023

FIT TIME: 0.00012
END: Sun Sep 17 21:25:25 2023

       class = 1      
START: Sun Sep 17 21:25:27 2023

FIT TIME: 0.000363
END: Sun Sep 17 21:25:27 2023

       class = 2      
START: Sun Sep 17 21:25:2

## Yacht gydrodinamics

Датасет "Yacht Hydrodynamics" содержит информацию о характеристиках моделей яхт и их гидродинамических свойствах.

Размерность решаемой задачи: 308 (число объектов) * 7 (число признаков)

Датасет содержит следующие признаки:

1. Longitudinal position of the center of buoyancy (положение продольного центра подъемной силы)
   - Диапазон значений: -5.0, 1.0

2. Prismatic coefficient (призматический коэффициент)
   - Диапазон значений: 0.41, 0.98

3. Length-displacement ratio (отношение длины к смещению)
   - Диапазон значений: 4.9, 6.5

4. Beam-draught ratio (отношение ширины к осадке)
   - Диапазон значений: 2.73, 4.43

5. Length-beam ratio (отношение длины к ширине)
   - Диапазон значений: 2.50, 9.65

6. Froude number (число Фруда)
   - Диапазон значений: 0.125, 0.824

7. Residuary resistance per unit weight of displacement (остаточное сопротивление на единицу массы смещения)
   - Диапазон значений: 0.0, 59.99

Первые 6 признаков будем подавать на вход алгоритмам, а 7 признак будем рассматривать в качестве целевой переменной. Таким образом, решается задача прогнозирования остаточного сопротивления парусных яхт на начальном этапе проектирования.

In [113]:
data = pd.read_csv('yacht_hydrodynamics.data', sep=' ', header=None)
data

,0,1,2,3,4,5,6
0,-2.3,0.568,4.78,3.99,3.17,0.125,0.11
1,-2.3,0.568,4.78,3.99,3.17,0.150,0.27
2,-2.3,0.568,4.78,3.99,3.17,0.175,0.47
3,-2.3,0.568,4.78,3.99,3.17,0.200,0.78
4,-2.3,0.568,4.78,3.99,3.17,0.225,1.18
...,...,...,...,...,...,...,...
303,-2.3,0.600,4.34,4.23,2.73,0.350,8.47
304,-2.3,0.600,4.34,4.23,2.73,0.375,12.27
305,-2.3,0.600,4.34,4.23,2.73,0.400,19.59
306,-2.3,0.600,4.34,4.23,2.73,0.425,30.48


Для кодирования вещественных признаков применяется процедура деления на интервалы. Выбирается 5 порогов для признака col, делящих обучающую выборку по этому признаку на 5 равных частей, после чего вещественные значения заменяются на категориальные интервалы.

In [114]:
for i, col in enumerate(data.columns[:-1]):
    data[col] = data[col].apply(lambda x: float(x))
    min_v = min(data[col])
    max_v = max(data[col])
    length = (max_v - min_v) / 5
    data[col] = data[col].apply(lambda x: round((x - min_v) / length))

In [115]:
data

,0,1,2,3,4,5,6
0,3,3,3,2,2,0,0.11
1,3,3,3,2,2,0,0.27
2,3,3,3,2,2,1,0.47
3,3,3,3,2,2,1,0.78
4,3,3,3,2,2,2,1.18
...,...,...,...,...,...,...,...
303,3,5,0,3,0,3,8.47
304,3,5,0,3,0,4,12.27
305,3,5,0,3,0,4,19.59
306,3,5,0,3,0,5,30.48


Закодируем категориальные признаки в числовые значения с помощью LabelEncoder, разделим данные на обучающую и тестовую выборки и будем хранить перебираемые 
далее значения гиперпараметров.


In [116]:
# Загрузка и подготовка данных
# data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data', 
                   # delim_whitespace=True, header=None)
X = data.iloc[:,:-1].values
y = data.iloc[:, -1].values


X_train_lab = copy(X)
le = LabelEncoder()
for j in range(X.shape[1]): # кодируем значения признаков
    X_train_lab[:, j] = le.fit_transform(X[:, j])
X_train, X_test, y_train, y_test = train_test_split(X_train_lab, y, test_size=0.2, random_state=42)


# # Scale the features
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# Задаем гиперпараметры для всех моделей
# lr_params = {'C': [0.001, 0.01, 0.05, 0.1, 0.15, 0.5, 1, 10]}
# gb_params = {'n_estimators': [50, 100, 150, 300, 500, 750, 1000, 1500, 2000], 'max_depth': [3, 5, 7, 10, 15]}
# knn_params = {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance']}
# rf_params = {'n_estimators': [14, 30, 50, 75, 100, 125, 150, 200, 300, 500], 'max_depth': [3, 5, 7, 10, 15]}
# svm_params = {'C': [0.001, 0.01, 0.05, 0.1, 0.15, 0.5, 1, 10], 'kernel': ['linear', 'rbf']}

lr_params = {'fit_intercept': [True, False]}
gb_params = {'n_estimators': [50, 100, 150], 'max_depth': [3, 5, 7]}
knn_params = {'n_neighbors': [3, 5, 7, 9, 11]}
rf_params = {'n_estimators': [50, 100, 150], 'max_depth': [3, 5, 7]}
svm_params = {'C': [0.001, 0.01, 0.1, 1, 10], 'kernel': ['linear', 'rbf']}
a1_params = {'eps': [0.1, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.2, 1.5, 1.7, 2.0],\
             'min_samples': [3, 5, 7, 10, 15, 30, 40, 50, 75, 100]\
            }



# Создаем объекты моделей и параметров восстановления
models = [LinearRegression(),\
          GradientBoostingRegressor(),\
          KNeighborsRegressor(),\
          RandomForestRegressor(),\
          SVR(),\
          A1_Rg()]

params = [lr_params,\
          gb_params,\
          knn_params,\
          rf_params,\
          svm_params,\
          a1_params]

In [117]:
X_train = X_train.astype(int)
X_test = X_test.astype(int)

In [118]:
np.unique(y_test)

array([3.000e-02, 4.000e-02, 5.000e-02, 8.000e-02, 1.000e-01, 1.500e-01,
       2.000e-01, 2.300e-01, 2.600e-01, 2.800e-01, 3.200e-01, 3.500e-01,
       3.600e-01, 4.000e-01, 4.600e-01, 4.700e-01, 5.500e-01, 6.600e-01,
       6.900e-01, 7.800e-01, 9.300e-01, 9.700e-01, 1.130e+00, 1.240e+00,
       1.280e+00, 1.370e+00, 1.670e+00, 1.760e+00, 1.820e+00, 1.830e+00,
       1.990e+00, 2.460e+00, 2.630e+00, 2.840e+00, 2.910e+00, 3.040e+00,
       3.350e+00, 3.430e+00, 3.450e+00, 3.700e+00, 3.760e+00, 3.990e+00,
       5.070e+00, 5.820e+00, 6.310e+00, 7.160e+00, 8.280e+00, 8.680e+00,
       8.760e+00, 1.199e+01, 1.215e+01, 1.377e+01, 1.424e+01, 1.959e+01,
       2.091e+01, 2.109e+01, 2.413e+01, 3.450e+01, 4.914e+01, 5.094e+01,
       5.517e+01])

In [119]:
y_train = y_train.astype(float)
y_test = y_test.astype(float)

In [120]:
X_train.shape

(246, 6)

In [121]:
y_train.shape

(246,)

In [122]:
# Создаем списки результатов и метрик времени работы
results = []
time_results = []

Подберем оптимальные значения параметров моделей с помощью GridSearchCV и оценим качество работы моделей с полученными параметрами на кросс-валидации по 5 фолдам: по очередности использовались 4 фолда для обучения и 1 фолд для контроля.

In [123]:
# Используем цикл для обучения и оценки каждой модели
for model, param in tqdm(zip(models, params)):
    start_time = time.time()
    # Используем GridSearchCV для поиска лучших параметров модели
    grid = GridSearchCV(model, param, cv=5, scoring='neg_mean_squared_error')
    # print("____")
    grid.fit(X_train, y_train)
    # Вычисляем среднеквадратичную ошибку на кросс-валидации
    # print("*****")
    scores = cross_val_score(grid.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    mse = np.abs(np.mean(scores))
    r2 = r2_score(y_test, grid.predict(X_test))    
    # Добавляем результаты в список
    results.append(f"{model.__class__.__name__}:\nBest params:{grid.best_params_}\nMSE: {mse:.2f}\nR2: {r2:.2f}\n")
    # Замеряем время работы
    end_time = time.time()
    time_results.append({'model': model.__class__.__name__, 'time': end_time - start_time})
    
# Выводим результаты
print('Results:')
for res in results:
    print(''.join(res))

print('\nTime results:')
for tres in time_results:
    print(tres)

5it [00:04,  1.06it/s]

       class = 0      
START: Sun Sep 17 21:36:02 2023

FIT TIME: 0.001288
END: Sun Sep 17 21:36:02 2023

       class = 1      
START: Sun Sep 17 21:36:04 2023

FIT TIME: 0.000106
END: Sun Sep 17 21:36:04 2023

       class = 2      
START: Sun Sep 17 21:36:06 2023

FIT TIME: 0.000127
END: Sun Sep 17 21:36:06 2023

       class = 3      
START: Sun Sep 17 21:36:08 2023

FIT TIME: 0.000248
END: Sun Sep 17 21:36:08 2023

       class = 4      
START: Sun Sep 17 21:36:10 2023

FIT TIME: 0.000212
END: Sun Sep 17 21:36:10 2023

       class = 5      
START: Sun Sep 17 21:36:12 2023

FIT TIME: 0.000216
END: Sun Sep 17 21:36:12 2023

       class = 6      
START: Sun Sep 17 21:36:14 2023

FIT TIME: 0.00011
END: Sun Sep 17 21:36:14 2023

       class = 7      
START: Sun Sep 17 21:36:16 2023

FIT TIME: 0.000206
END: Sun Sep 17 21:36:16 2023

       class = 8      
START: Sun Sep 17 21:36:18 2023

FIT TIME: 0.00019
END: Sun Sep 17 21:36:18 2023

       class = 9      
START: Sun Sep 17 21:36:1

/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k


START: Sun Sep 17 21:37:59 2023

       class = 0      


/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k


...

START: Sun Sep 17 22:31:32 2023

       class = 0      


/tmp/ipykernel_4723/1656538666.py:78: RuntimeWarning: invalid value encountered in true_divide
  counter[:, i] /= k


START: Sun Sep 17 22:31:34 2023

FIT TIME: 0.002541
END: Sun Sep 17 22:31:34 2023

       class = 1      
START: Sun Sep 17 22:31:36 2023

FIT TIME: 0.000426
END: Sun Sep 17 22:31:36 2023

       class = 2      
START: Sun Sep 17 22:31:38 2023

FIT TIME: 0.000934
END: Sun Sep 17 22:31:38 2023

       class = 3      
START: Sun Sep 17 22:31:40 2023

FIT TIME: 0.000388
END: Sun Sep 17 22:31:40 2023

       class = 4      
START: Sun Sep 17 22:31:42 2023

FIT TIME: 0.000934
END: Sun Sep 17 22:31:42 2023

       class = 5      
START: Sun Sep 17 22:31:44 2023

FIT TIME: 0.000316
END: Sun Sep 17 22:31:44 2023

       class = 6      
START: Sun Sep 17 22:31:46 2023

FIT TIME: 0.000217
END: Sun Sep 17 22:31:46 2023

       class = 7      
START: Sun Sep 17 22:31:48 2023

FIT TIME: 0.00036
END: Sun Sep 17 22:31:48 2023

       class = 8      
START: Sun Sep 17 22:31:50 2023

FIT TIME: 0.000387
END: Sun Sep 17 22:31:50 2023

       class = 9      
START: Sun Sep 17 22:31:52 2023

FIT TIME: 0.00

6it [56:59, 569.86s/it] 

Results:
LinearRegression:
Best params:{'fit_intercept': True}
MSE: 86.26
R2: 0.53

GradientBoostingRegressor:
Best params:{'max_depth': 3, 'n_estimators': 50}
MSE: 31.99
R2: 0.87

KNeighborsRegressor:
Best params:{'n_neighbors': 7}
MSE: 54.31
R2: 0.66

RandomForestRegressor:
Best params:{'max_depth': 3, 'n_estimators': 100}
MSE: 23.56
R2: 0.92

SVR:
Best params:{'C': 10, 'kernel': 'rbf'}
MSE: 39.06
R2: 0.84

A1_Rg:
Best params:{'eps': 1.0, 'min_samples': 7}
MSE: 108.77
R2: 0.47


Time results:
{'model': 'LinearRegression', 'time': 0.04763364791870117}
{'model': 'GradientBoostingRegressor', 'time': 1.1359622478485107}
{'model': 'KNeighborsRegressor', 'time': 0.04797101020812988}
{'model': 'RandomForestRegressor', 'time': 3.4772820472717285}
{'model': 'SVR', 'time': 0.1517183780670166}
{'model': 'A1_Rg', 'time': 3414.295716524124}


Оценим качество моделей с оптимальными параметрами на тестовой выборке подсчетом метрик RMSE и R2 score.

In [124]:
model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("LinearRegression:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = GradientBoostingRegressor(max_depth=3, n_estimators=50)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("GradientBoostingRegressor:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = KNeighborsRegressor(n_neighbors=7)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("KNeighborsRegressor:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = RandomForestRegressor(max_depth=3, n_estimators=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("RandomForestRegressor:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = SVR(C=10, kernel='rbf')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("SVR:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

model = A1_Rg(eps=1.0, min_samples=7)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("A1_Rg:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

LinearRegression:
Mean Squared Error: 8.361844658109183
R2 Score: 0.529430982888861
GradientBoostingRegressor:
Mean Squared Error: 4.432378003436255
R2 Score: 0.8677813493337565
KNeighborsRegressor:
Mean Squared Error: 7.072174615023664
R2 Score: 0.6633914919121962
RandomForestRegressor:
Mean Squared Error: 3.597944144090632
R2 Score: 0.9128779727172105
SVR:
Mean Squared Error: 4.878148138254081
R2 Score: 0.8398492015294785
       class = 0      
START: Sun Sep 17 22:49:26 2023

FIT TIME: 0.001232
END: Sun Sep 17 22:49:26 2023

       class = 1      
START: Sun Sep 17 22:49:29 2023

FIT TIME: 0.000195
END: Sun Sep 17 22:49:29 2023

       class = 2      
START: Sun Sep 17 22:49:31 2023

FIT TIME: 0.000975
END: Sun Sep 17 22:49:31 2023

       class = 3      
START: Sun Sep 17 22:49:33 2023

FIT TIME: 0.000227
END: Sun Sep 17 22:49:33 2023

       class = 4      
START: Sun Sep 17 22:49:36 2023

FIT TIME: 0.000972
END: Sun Sep 17 22:49:36 2023

       class = 5      
START: Sun Sep 17 2

Еще раз протестируем полученные модели и оценим их качество и устойчивость работы, усредняя результаты подсчета метрик и времени работы по 10 независимым запускам на случайных разбиениях данных.


In [125]:
res_rmse = [0.0] * 6
tres = [0.0] * 6
res_r2 = [0.0] * 6
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_train_lab, y, test_size=0.2, random_state=42 + i)
    X_train = X_train.astype(int)
    X_test = X_test.astype(int)
    y_train = y_train.astype(float)
    y_test = y_test.astype(float)
    
    start = time.time()
    model = LinearRegression(fit_intercept=True)
    model.fit(X_train, y_train)
    res_rmse[0] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[0] += r2_score(y_test, model.predict(X_test))
    tres[0] += time.time() - start
    
    start = time.time()
    model = GradientBoostingRegressor(max_depth=3, n_estimators=50)
    model.fit(X_train, y_train)
    res_rmse[1] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[1] += r2_score(y_test, model.predict(X_test))
    tres[1] += time.time() - start
    
    start = time.time()
    model = KNeighborsRegressor(n_neighbors=7)
    model.fit(X_train, y_train)
    res_rmse[2] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[2] += r2_score(y_test, model.predict(X_test))
    tres[2] += time.time() - start
    
    start = time.time()
    model = RandomForestRegressor(max_depth=3, n_estimators=100)
    model.fit(X_train, y_train)
    res_rmse[3] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[3] += r2_score(y_test, model.predict(X_test))
    tres[3] += time.time() - start
    
    start = time.time()
    model = SVR(C=10, kernel='rbf')
    model.fit(X_train, y_train)
    res_rmse[4] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[4] += r2_score(y_test, model.predict(X_test))
    tres[4] += time.time() - start
    
    start = time.time()
    model = A1_Rg(eps=1.0, min_samples=7)
    model.fit(X_train, y_train)
    res_rmse[5] += mean_squared_error(y_test, model.predict(X_test), squared=False)
    res_r2[5] += r2_score(y_test, model.predict(X_test))
    tres[5] += time.time() - start
    
print("LinearRegression:")
print("Mean Squared Error:", res_rmse[0] / 10)
print("R2 Score:", res_r2[0] / 10)
print("Time:", tres[0] / 10)

print("GradientBoostingRegressor:")
print("Mean Squared Error:", res_rmse[1] / 10)
print("R2 Score:", res_r2[1] / 10)
print("Time:", tres[1] / 10)

print("KNeighborsRegressor:")
print("Mean Squared Error:", res_rmse[2] / 10)
print("R2 Score:", res_r2[2] / 10)
print("Time:", tres[2] / 10)

print("RandomForestRegressor:")
print("Mean Squared Error:", res_rmse[3] / 10)
print("R2 Score:", res_r2[3] / 10)
print("Time:", tres[3] / 10)

print("SVR:")
print("Mean Squared Error:", res_rmse[4] / 10)
print("R2 Score:", res_r2[4] / 10)
print("Time:", tres[4] / 10)

print("A1_Rg:")
print("Mean Squared Error:", res_rmse[5] / 10)
print("R2 Score:", res_r2[5] / 10)
print("Time:", tres[5] / 10)

       class = 0      
START: Sun Sep 17 22:52:02 2023

FIT TIME: 0.002345
END: Sun Sep 17 22:52:02 2023

       class = 1      
START: Sun Sep 17 22:52:05 2023

FIT TIME: 0.000253
END: Sun Sep 17 22:52:05 2023

       class = 2      
START: Sun Sep 17 22:52:07 2023

FIT TIME: 0.001003
END: Sun Sep 17 22:52:07 2023

       class = 3      
START: Sun Sep 17 22:52:09 2023

FIT TIME: 0.000411
END: Sun Sep 17 22:52:09 2023

       class = 4      
START: Sun Sep 17 22:52:11 2023

FIT TIME: 0.001044
END: Sun Sep 17 22:52:11 2023

       class = 5      
START: Sun Sep 17 22:52:13 2023

FIT TIME: 0.000296
END: Sun Sep 17 22:52:13 2023

       class = 6      
START: Sun Sep 17 22:52:15 2023

FIT TIME: 0.000459
END: Sun Sep 17 22:52:15 2023

       class = 7      
START: Sun Sep 17 22:52:17 2023

FIT TIME: 0.000214
END: Sun Sep 17 22:52:17 2023

       class = 8      
START: Sun Sep 17 22:52:19 2023

FIT TIME: 0.000465
END: Sun Sep 17 22:52:19 2023

       class = 9      
START: Sun Sep 17 22:52